# On Sale - ECD I

### Takeways - 02/10/2022

Firstly I need to know if the score on the leaderboard is equivalent to the accuracy here. If so, the professor wants us to overfit the data. If not, how can I know if I am being good enough to post my results on kaggle?
*R: my accuracy here is the actual score on the leaderboard!*

Ideas of models/changes that I can implement and submit:
1. Decision Tree Classifier
2. Decision Tree Classifier w/ different missing values imputation
3. Logistic Regression for Classification Task
4. Random Forest for Classification Task
5. XGBoost for Classification Task (learn more)
6. Ensemble Model Stacking (learn more)

### Takeways - 04/10/2022

NaN values - I should check the min/max and quartiles of the continuous variables. If I have outliers, I could use the median as input instead of the mean

Continuous variables - I could use encoders to transform them into discrete variables and see the results

Algorithm - attention to the hyperparameters!

Variable Selection -  keep only the variables that are significant to our analysis

IMPORTANT - the variable w/ missing values is a continuous variables! -> variable x9

## First Step - Deal with NaN Values and Encoders

In [304]:
#Change directory

import pandas as pd
import numpy as np
import os

os.chdir("//Users//enzoaaragao//Downloads//on-sale-ecd1")

In [305]:
train = pd.read_csv("training.csv")
test = pd.read_csv("task.csv")

In [169]:
train.y.value_counts()

0    2920
1     756
Name: y, dtype: int64

------------

### 1.1. Dealing w/ NaN values

In [7]:
train.isna().any()

i      False
y      False
d1     False
d2     False
d3     False
d4     False
d5     False
d7     False
d8     False
d9     False
d10    False
d11    False
d12    False
d13    False
d14    False
d15    False
d16    False
d17    False
d18    False
x1     False
x2     False
x3     False
x4     False
x5     False
x6     False
x7     False
x8     False
x9      True
dtype: bool

##### The column "x9" has NaN values

In [10]:
#Check if the "x9" column has a minimal value of zero - in this way could be a ab opportunity to replace the NaN values w/ zero
train["x9"].describe()

#Nevertheless, we see that the minimal value of one

count    3627.000000
mean       26.316515
std        15.518175
min         1.000000
25%        13.500000
50%        28.000000
75%        40.000000
max        54.000000
Name: x9, dtype: float64

In [13]:
print("the mean value is", train["x9"].mean())
print(" ")
print("the median value is", train["x9"].median())
print(" ")
print("the mode value is", train["x9"].mode())

the mean value is 26.316515026192445
 
the median value is 28.0
 
the mode value is 0    2.0
Name: x9, dtype: float64


As we can see, the mode significantly diverges from the mean and median. On the other hand, the mean and median are close from each other

https://towardsdatascience.com/whats-the-best-way-to-handle-nan-values-62d50f738fc --- the problem with using the mean, mode, or median values of the column as input for the NaN values only takes into account the value of the respective column. Nevertheless, I can use methods such as KNN and MICE that takes into account the whole dataset to input values

In [306]:
X_train=train.iloc[:,2:len(train.columns)]
y_train=train.iloc[:,1]
X_test=test.iloc[:,2:len(test.columns)]

In [307]:
#Iterative Imputer - MICE (I can also do w/ KNN)

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
imp = IterativeImputer(max_iter=1000, random_state=0)
X_train=imp.fit_transform(X_train)
X_test=imp.fit_transform(X_test)

In [308]:
#Standar Scaler

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

I will think about the encoders later on -> now I am skipping to the Algo part

### 1.2. Quartile Encorders of Continuous Variables

In [296]:
X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)
#X_train.describe()

#17 = 1 to 2; 2 to 4; 4 to 6; 6 to 7
#18 = 1 to 1; 1 to 2; 2 to 3; 3 to 4
#19 = 0.001 to 0.350; 0.350 to 0.500; 0.500 to 0.750; 0.750 to 5
#20 = 1 to 1; 1 to 1; 1 to 1; 1 to 3
#21 = 0.39 to 1.09; 1.09 to 1.69: 1.69 to 2.82; 2.82 to 27.48
#22 = 1 to 1; 1 to 5; 5 to 8; 8 to 17
#23 = 1 to 4; 4 to 6; 6 to 12; 12 to 25
#24 = 1 to 3; 3 to 4; 4 to 5; 5 to 12
#25 = 1 to 14: 14 to 27.90; 27.90 to 40; 40 to 54

X_train[17] = pd.cut(X_train[17], bins=4, labels = [1,2,3,4])
X_train[18] = pd.cut(X_train[18], bins=3, labels = [1,2,3])
X_train[19] = pd.cut(X_train[19], bins=4, labels = [1,2,3,4])
X_train[20] = pd.cut(X_train[20], bins=2, labels = [1,2])
X_train[21] = pd.cut(X_train[21], bins=4, labels = [1,2,3,4])
X_train[22] = pd.cut(X_train[22], bins=3, labels = [1,2,3])
X_train[23] = pd.cut(X_train[23], bins=4, labels = [1,2,3,4])
X_train[24] = pd.cut(X_train[24], bins=4, labels = [1,2,3,4])
X_train[25] = pd.cut(X_train[25], bins=4, labels = [1,2,3,4])

X_test[17] = pd.cut(X_test[17], bins=4, labels = [1,2,3,4])
X_test[18] = pd.cut(X_test[18], bins=3, labels = [1,2,3])
X_test[19] = pd.cut(X_test[19], bins=4, labels = [1,2,3,4])
X_test[20] = pd.cut(X_test[20], bins=2, labels = [1,2])
X_test[21] = pd.cut(X_test[21], bins=4, labels = [1,2,3,4])
X_test[22] = pd.cut(X_test[22], bins=3, labels = [1,2,3])
X_test[23] = pd.cut(X_test[23], bins=4, labels = [1,2,3,4])
X_test[24] = pd.cut(X_test[24], bins=4, labels = [1,2,3,4])
X_test[25] = pd.cut(X_test[25], bins=4, labels = [1,2,3,4]) 

In [107]:
X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)
X_train

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2,1,1,1,1,3,1,2,3
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1,2,1,1,1,1,1,2,3
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2,2,1,1,1,1,1,2,3
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,2,1,1,1,3,1,2,1,3
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2,2,1,1,1,1,1,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3671,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1,1,1,1,1,1,4,2,3
3672,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1,1,1,1,1,1,2,2,2
3673,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,3,1,1,1,1,1,1,2,1
3674,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,2,1,1,1,1,3,1,1,2


--------------

## Second Step - Algorithm and Parameters

### 2.1. Algorithms

In [176]:
#Logistic Regression

from sklearn.linear_model import LogisticRegression
reg = LogisticRegression()
reg.fit(X_train, y_train)
y_pred_lr = reg.predict(X_test)

In [177]:
from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(y_train,y_pred_lr))

              precision    recall  f1-score   support

           0       0.80      0.95      0.87      2920
           1       0.26      0.07      0.11       756

    accuracy                           0.77      3676
   macro avg       0.53      0.51      0.49      3676
weighted avg       0.69      0.77      0.71      3676



In [178]:
#Decision Tree Classifier

from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier()
dtc.fit(X_train, y_train)
y_pred_dtc = dtc.predict(X_test)

In [179]:
from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(y_train,y_pred_dtc))

              precision    recall  f1-score   support

           0       0.80      0.79      0.80      2920
           1       0.21      0.21      0.21       756

    accuracy                           0.67      3676
   macro avg       0.50      0.50      0.50      3676
weighted avg       0.68      0.67      0.68      3676



In [209]:
#Random Forest

from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)
y_pred_rf = rfc.predict(X_test)

In [210]:
from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(y_train,y_pred_rf))

              precision    recall  f1-score   support

           0       0.80      0.81      0.80      2920
           1       0.21      0.19      0.20       756

    accuracy                           0.68      3676
   macro avg       0.50      0.50      0.50      3676
weighted avg       0.68      0.68      0.68      3676



In [182]:
#Support Vector Machine (SVM)

from sklearn.svm import SVC
svc = SVC()
svc.fit(X_train, y_train)
y_pred_svm = svc.predict(X_test)

In [183]:
from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(y_train,y_pred_svm))

              precision    recall  f1-score   support

           0       0.80      0.94      0.86      2920
           1       0.23      0.07      0.10       756

    accuracy                           0.76      3676
   macro avg       0.51      0.50      0.48      3676
weighted avg       0.68      0.76      0.71      3676



In [184]:
#K-Nearest Neighbour (KNN)

from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
y_pred_knn = knn.predict(X_test)

In [185]:
from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(y_train,y_pred_knn))

              precision    recall  f1-score   support

           0       0.80      0.84      0.82      2920
           1       0.23      0.19      0.21       756

    accuracy                           0.71      3676
   macro avg       0.52      0.51      0.51      3676
weighted avg       0.68      0.71      0.69      3676



In [186]:
#Naive Bayes

from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_train, y_train)
y_pred_nb = gnb.predict(X_test)

In [187]:
from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(y_train,y_pred_nb))

              precision    recall  f1-score   support

           0       0.79      0.53      0.64      2920
           1       0.20      0.44      0.27       756

    accuracy                           0.52      3676
   macro avg       0.49      0.49      0.45      3676
weighted avg       0.67      0.52      0.56      3676



In [188]:
### TESTE
#Model Pipeline

model_pipeline = []
model_pipeline.append(LogisticRegression(solver='liblinear'))
model_pipeline.append(SVC())
model_pipeline.append(KNeighborsClassifier())
model_pipeline.append(DecisionTreeClassifier())
model_pipeline.append(RandomForestClassifier())
model_pipeline.append(GaussianNB())

In [189]:
### TESTE
#Model Evaluation

from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix

model_list = ['Logistic Regression', 'SVM', 'KNN', 'Decision Tree', 'Random Forest', 'Naive Bayes']
acc_list = []
auc_list = []
cm_list = []

for model in model_pipeline:
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc_list.append(metrics.accuracy_score(y_train, y_pred))
    fpr, tpr, _thresholds = metrics.roc_curve(y_train, y_pred)
    auc_list.append(round(metrics.auc(fpr, tpr),2))
    cm_list.append(confusion_matrix(y_train, y_pred))

In [190]:
### TESTE
#Accuracy and AUC

result_df_cont = pd.DataFrame({'Model':model_list, 'Accuracy': acc_list, 'AUC': auc_list})
result_df_cont

,Model,Accuracy,AUC
0,Logistic Regression,0.770131,0.51
1,SVM,0.760881,0.50
2,KNN,0.705658,0.51
3,Decision Tree,0.673558,0.50
4,Random Forest,0.684712,0.51
5,Naive Bayes,0.515506,0.49


In [191]:
result_df_d = pd.DataFrame({'Model':model_list, 'Accuracy': acc_list, 'AUC': auc_list})
result_df_d

,Model,Accuracy,AUC
0,Logistic Regression,0.770131,0.51
1,SVM,0.760881,0.50
2,KNN,0.705658,0.51
3,Decision Tree,0.673558,0.50
4,Random Forest,0.684712,0.51
5,Naive Bayes,0.515506,0.49


In [100]:
### Decision Tree Classifier w/ Encoders - Teste 1

#Generate CSV file for Submission
res = pd.DataFrame(data={'i':test.iloc[:,0], 'y': y_pred_dtc})
res.to_csv("//Users//enzoaaragao//Downloads//on-sale-ecd1//dtc_cat_1.csv",index=False)

# Nota baixa: 0.84

In [ ]:
### Decision Tree Classifier w/ Encoders - Teste 1

#Generate CSV file for Submission
res = pd.DataFrame(data={'i':test.iloc[:,0], 'y': y_pred_dtc})
res.to_csv("//Users//enzoaaragao//Downloads//on-sale-ecd1//dtc_cat_1.csv",index=False)

# Nota: 0.84

In [101]:
### Random Forest w/ Encoders - Teste 1

#Generate CSV file for Submission
res = pd.DataFrame(data={'i':test.iloc[:,0], 'y': y_pred_rf})
res.to_csv("//Users//enzoaaragao//Downloads//on-sale-ecd1//rf_cat_1.csv",index=False)

# Nota: 0.84
# Random Forest w/ MICE Nan Values and Quantile Encoder

In [111]:
### Naive Bayes w/ Encoders - Teste 1

#Generate CSV file for Submission
res = pd.DataFrame(data={'i':test.iloc[:,0], 'y': y_pred_nb})
res.to_csv("//Users//enzoaaragao//Downloads//on-sale-ecd1//nb_cat_1.csv",index=False)

# Nota: 0.36
# Naive Bayes w/ MICE Nan Values and Quantile Encoder

In [114]:
### SVM w/ Encoders - Teste 1

#Generate CSV file for Submission
res = pd.DataFrame(data={'i':test.iloc[:,0], 'y': y_pred_svm})
res.to_csv("//Users//enzoaaragao//Downloads//on-sale-ecd1//svm_cat_1.csv",index=False)

# Nota: 0.80
# SVM w/ MICE Nan Values and Quantile Encoder

In [117]:
### LR w/ Encoders - Teste 1

#Generate CSV file for Submission
res = pd.DataFrame(data={'i':test.iloc[:,0], 'y': y_pred_lr})
res.to_csv("//Users//enzoaaragao//Downloads//on-sale-ecd1//lr_cat_1.csv",index=False)

# Nota: 0.81
# LR w/ MICE Nan Values and Quantile Encoder

In [125]:
### Decision Tree Classifier - Teste 2

#Generate CSV file for Submission
res = pd.DataFrame(data={'i':test.iloc[:,0], 'y': y_pred_dtc})
res.to_csv("//Users//enzoaaragao//Downloads//on-sale-ecd1//dtc_1.csv",index=False)

# Nota: 0.947
# DTC w/ MICE Nan Values Max_iter = 500

In [134]:
### Random Forest - Teste 2

#Generate CSV file for Submission
res = pd.DataFrame(data={'i':test.iloc[:,0], 'y': y_pred_rf})
res.to_csv("//Users//enzoaaragao//Downloads//on-sale-ecd1//rf_1.csv",index=False)

# Nota: 0.954
# Random Forest w/ MICE Nan Values Max_iter = 1000

In [137]:
### SVM - Teste 2

#Generate CSV file for Submission
res = pd.DataFrame(data={'i':test.iloc[:,0], 'y': y_pred_svm})
res.to_csv("//Users//enzoaaragao//Downloads//on-sale-ecd1//svm_1.csv",index=False)

# Nota: 0.
# SVM w/ MICE Nan Values Max_iter = 1000

In [140]:
### Naive Bayes - Teste 2

#Generate CSV file for Submission
res = pd.DataFrame(data={'i':test.iloc[:,0], 'y': y_pred_nb})
res.to_csv("//Users//enzoaaragao//Downloads//on-sale-ecd1//nb_1.csv",index=False)

# Nota: 0.63
# NB w/ MICE Nan Values Max_iter = 1000

In [211]:
### Random Forest - Teste 3

#Generate CSV file for Submission
res = pd.DataFrame(data={'i':test.iloc[:,0], 'y': y_pred_rf})
res.to_csv("//Users//enzoaaragao//Downloads//on-sale-ecd1//rf_2.csv",index=False)

# Nota: 0.950
# RF Scaled w/ MICE Nan Values Max_iter = 1000

In [231]:
from sklearn.ensemble import BaggingClassifier

bag_model = BaggingClassifier(
    base_estimator = DecisionTreeClassifier(),
    n_estimators = 100,
    max_samples=0.8,
    oob_score=True,
)

bag_model.fit(X_train, y_train)
y_pred_bag = bag_model.predict(X_test)

In [232]:
from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(y_train,y_pred_bag))

              precision    recall  f1-score   support

           0       0.80      0.80      0.80      2920
           1       0.21      0.21      0.21       756

    accuracy                           0.68      3676
   macro avg       0.50      0.50      0.50      3676
weighted avg       0.68      0.68      0.68      3676



# BINGO 1

In [233]:
### DTC Bag Model - Teste 3 *** BINGO 1

#Generate CSV file for Submission
res = pd.DataFrame(data={'i':test.iloc[:,0], 'y': y_pred_bag})
res.to_csv("//Users//enzoaaragao//Downloads//on-sale-ecd1//dtc_bag_3.csv",index=False)

# Nota: 0.954
# DTC Bag w/ MICE Nan Values Max_iter = 1000

In [215]:
from sklearn.ensemble import BaggingClassifier

bag_model = BaggingClassifier(
    base_estimator = RandomForestClassifier(),
    n_estimators = 1000,
    max_samples=0.9,
    oob_score=True,
)

bag_model.fit(X_train, y_train)
y_pred_bag_rf = bag_model.predict(X_test)

In [216]:
from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(y_train,y_pred_bag_rf))

              precision    recall  f1-score   support

           0       0.80      0.83      0.81      2920
           1       0.21      0.18      0.19       756

    accuracy                           0.69      3676
   macro avg       0.50      0.50      0.50      3676
weighted avg       0.68      0.69      0.68      3676



In [217]:
### RF Bag Model - Teste 4

#Generate CSV file for Submission
res = pd.DataFrame(data={'i':test.iloc[:,0], 'y': y_pred_bag_rf})
res.to_csv("//Users//enzoaaragao//Downloads//on-sale-ecd1//rf_bag_1.csv",index=False)

# Nota: 0.
# RF Bag Scaled w/ MICE Nan Values Max_iter = 1000

--------------------

### 2.2. Hyperparameters

In [192]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn import metrics

In [193]:
RandomForestClassifier().get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [159]:
### Random Forest Hyperparameters - Teste 3

#Generate CSV file for Submission
res = pd.DataFrame(data={'i':test.iloc[:,0], 'y': y_pred_hrf})
res.to_csv("//Users//enzoaaragao//Downloads//on-sale-ecd1//hrf_1.csv",index=False)

# Nota: 0.
# Hyper Random Forest w/ MICE Nan Values 

In [221]:
# GridSearchCV
n_estimators_list = [130,160,190]
criterion_list = ['gini']
max_depth_list = [20,55]
min_samples_split_list = [0.001,0.005]
min_samples_leaf_list = [0.001,0.005]
max_features_list = ['log2']

params_grid = {
    'n_estimators':n_estimators_list,
    'criterion':criterion_list,
    'max_depth':max_depth_list,
    'min_samples_split':min_samples_split_list,
    'min_samples_leaf':min_samples_leaf_list,
    'max_features':max_features_list
}

num_combinations = 1
for k in params_grid.keys(): num_combinations *= len(params_grid[k])
    
print('Number of combinations =', num_combinations)
params_grid

Number of combinations = 24


{'n_estimators': [130, 160, 190],
 'criterion': ['gini'],
 'max_depth': [35, 55],
 'min_samples_split': [0.001, 0.005],
 'min_samples_leaf': [0.001, 0.005],
 'max_features': ['log2']}

In [ ]:
def my_roc_auc_score(model, X, y): return metrics.roc_auc_score(y_train, model.predict(X_train))

model_rf = GridSearchCV(estimator=RandomForestClassifier(class_weight='balanced'),
                              param_grid=params_grid,
                              cv=2,
                              scoring=my_roc_auc_score,
                              return_train_score=True,
                              verbose=4)

model_rf.fit(X_train,y_train)

In [223]:
model_rf.best_params_

{'criterion': 'gini',
 'max_depth': 35,
 'max_features': 'log2',
 'min_samples_leaf': 0.001,
 'min_samples_split': 0.001,
 'n_estimators': 190}

In [252]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators = 130,
 min_samples_split = 0.0001,
 min_samples_leaf = 0.0001,
 max_features = 'sqrt',
 max_depth = 40,
 criterion = 'gini')
rfc.fit(X_train, y_train)
y_pred_hrf = rfc.predict(X_test)

In [250]:
from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(y_train,y_pred_hrf))

              precision    recall  f1-score   support

           0       0.80      0.81      0.80      2920
           1       0.21      0.20      0.20       756

    accuracy                           0.68      3676
   macro avg       0.50      0.50      0.50      3676
weighted avg       0.68      0.68      0.68      3676



In [226]:
### Random Forest Hyperparameters - Teste 4

#Generate CSV file for Submission
res = pd.DataFrame(data={'i':test.iloc[:,0], 'y': y_pred_hrf})
res.to_csv("//Users//enzoaaragao//Downloads//on-sale-ecd1//hrf_3.csv",index=False)

# Nota: 0.90
# Hyper Random Forest Scaled w/ MICE Nan Values 

In [253]:
### Random Forest Hyperparameters - Teste 10

#Generate CSV file for Submission
res = pd.DataFrame(data={'i':test.iloc[:,0], 'y': y_pred_hrf})
res.to_csv("//Users//enzoaaragao//Downloads//on-sale-ecd1//hrf_10.csv",index=False)

# Nota: 0.
# Hyper Random Forest w/ MICE Nan Values 

In [277]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier

xgb_cl = xgb.XGBClassifier(objectiver='binary:logistic', n_estimators=50, scale_pos_weight=10)
xgb_cl.fit(X_train, y_train, verbose=True)
y_pred_xgb = xgb_cl.predict(X_test)

[01:04:55] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_44tbtwf8c1/croots/recipe/xgboost-split_1659548960882/work/src/learner.cc:576: 
Parameters: { "objectiver" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[01:04:55] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_44tbtwf8c1/croots/recipe/xgboost-split_1659548960882/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [278]:
from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(y_train,y_pred_xgb))

              precision    recall  f1-score   support

           0       0.79      0.73      0.76      2920
           1       0.21      0.27      0.23       756

    accuracy                           0.63      3676
   macro avg       0.50      0.50      0.50      3676
weighted avg       0.67      0.63      0.65      3676



In [279]:
### XGBoost - Teste 1

xgb_cl = xgb.XGBClassifier(objectiver='binary:logistic', n_estimators=50, scale_pos_weight=10)
xgb_cl.fit(X_train, y_train, verbose=True)
y_pred_xgb = xgb_cl.predict(X_test)

#Generate CSV file for Submission
res = pd.DataFrame(data={'i':test.iloc[:,0], 'y': y_pred_xgb})
res.to_csv("//Users//enzoaaragao//Downloads//on-sale-ecd1//xgb_2.csv",index=False)

# Nota: 0.914
# XGBoost w/ MICE Nan Values T1

/Users/enzoaaragao/opt/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[01:04:59] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_44tbtwf8c1/croots/recipe/xgboost-split_1659548960882/work/src/learner.cc:576: 
Parameters: { "objectiver" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[01:04:59] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_44tbtwf8c1/croots/recipe/xgboost-split_1659548960882/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


# BINGO 2 - XGBOOST HYPERPARAMETER

In [ ]:
# GridsearchCV

#gbc = GradientBoostingClassifier()

params = {'learning_rate':[0.001, 0.01, 0.1, 0.25, 0.5, 0.4],
         'max_depth': [1,2,3,4,5,6,7,8,9,10],
         'max_features':[1,2,3,4,5,6,7,8,9,10],
         'n_estimators':[20,40,50,70,100]
}

model_xgb = GridSearchCV(XGBClassifier(),
                          param_grid=params,
                          cv=4,
                          n_jobs=1,
                         verbose=0)

model_xgb.fit(X_train,y_train)

In [310]:
model_xgb.best_params_

{'learning_rate': 0.25,
 'max_depth': 10,
 'max_features': 1,
 'n_estimators': 130}

In [288]:
xgb_cl = xgb.XGBClassifier(n_estimators=100, learning_rate=0.25, max_depth=8, scale_pos_weight=10, objectiver='binary:logistic')
xgb_cl.fit(X_train, y_train, verbose=True)
y_pred_xgb = xgb_cl.predict(X_test)

/Users/enzoaaragao/opt/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[01:14:56] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_44tbtwf8c1/croots/recipe/xgboost-split_1659548960882/work/src/learner.cc:576: 
Parameters: { "objectiver" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[01:14:56] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_44tbtwf8c1/croots/recipe/xgboost-split_1659548960882/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [275]:
### XGBoost - Teste 3

xgb_cl = xgb.XGBClassifier(n_estimators=100, learning_rate=0.25, max_depth=8)
xgb_cl.fit(X_train, y_train, verbose=True)
y_pred_xgb = xgb_cl.predict(X_test)

#Generate CSV file for Submission
res = pd.DataFrame(data={'i':test.iloc[:,0], 'y': y_pred_xgb})
res.to_csv("//Users//enzoaaragao//Downloads//on-sale-ecd1//xgb_h_3.csv",index=False)

# Nota: 0.
# Hyper XGBoost w/ MICE Nan Values

In [289]:
### XGBoost - Teste 5

xgb_cl = xgb.XGBClassifier(n_estimators=100, learning_rate=0.25, max_depth=8)
xgb_cl.fit(X_train, y_train, verbose=True)
y_pred_xgb = xgb_cl.predict(X_test)

#Generate CSV file for Submission
res = pd.DataFrame(data={'i':test.iloc[:,0], 'y': y_pred_xgb})
res.to_csv("//Users//enzoaaragao//Downloads//on-sale-ecd1//xgb_h_5.csv",index=False)

# Nota: 0.
# Hyper XGBoost w/ MICE Nan Values

[01:15:19] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_44tbtwf8c1/croots/recipe/xgboost-split_1659548960882/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [282]:
### XGBoost - Teste 4

#Generate CSV file for Submission
res = pd.DataFrame(data={'i':test.iloc[:,0], 'y': y_pred_xgb})
res.to_csv("//Users//enzoaaragao//Downloads//on-sale-ecd1//xgb_h_4.csv",index=False)

# Nota: 0.
# Hyper XGBoost w/ MICE Nan Values

In [311]:
### XGBoost - Teste 6

#learning_rate': 0.25,
# 'max_depth': 10,
# 'max_features': 1,
# 'n_estimators': 130

xgb_cl = xgb.XGBClassifier(n_estimators=130, learning_rate=0.25, max_depth = 10, max_features = 1)
xgb_cl.fit(X_train, y_train, verbose=True)
y_pred_xgb = xgb_cl.predict(X_test)

#Generate CSV file for Submission
res = pd.DataFrame(data={'i':test.iloc[:,0], 'y': y_pred_xgb})
res.to_csv("//Users//enzoaaragao//Downloads//on-sale-ecd1//xgb_h_6.csv",index=False)

# Nota: 0.
# Hyper XGBoost w/ MICE Nan Values & Standard - Teste 6

[02:34:23] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_44tbtwf8c1/croots/recipe/xgboost-split_1659548960882/work/src/learner.cc:576: 
Parameters: { "max_features" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[02:34:23] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_44tbtwf8c1/croots/recipe/xgboost-split_1659548960882/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
